<a href="https://colab.research.google.com/github/super-onigiri-man/ITE3-2/blob/main/FM_%E3%83%99%E3%82%B9%E3%83%88%E3%83%92%E3%83%83%E3%83%88%E3%83%A9%E3%83%B3%E3%82%AD%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd

def selecter():

    print("見たい週を選択してください")
    select = int(input("1:今回 2:前回 3:別の回"))
    return select

def OriconTodays():
    # 今日の日付を求める
    d_today = datetime.date.today()
    # 今日の曜日を求める
    todayweek = datetime.date.today().weekday()
    #print(d_today, todayweek)

    #オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    #日付は来週の水曜日付となる。
    if (todayweek == 0):  # 今日が月曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today
    elif (todayweek == 1):# 今日が火曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today - datetime.timedelta(days=1)
    elif (todayweek == 2):# 今日が水曜日
        Oriconday = d_today + datetime.timedelta(days=5)
    elif (todayweek == 3):# 今日が木曜日
        Oriconday = d_today + datetime.timedelta(days=4)
    elif (todayweek == 4):# 今日が金曜日
        Oriconday = d_today + datetime.timedelta(days=3)
    elif (todayweek == 5):# 今日が土曜日
        Oriconday = d_today + datetime.timedelta(days=2)
    elif (todayweek == 6):# 今日が日曜日
        Oriconday = d_today + datetime.timedelta(days=1)

    return Oriconday


def OriconLastWeek():
    # 今日の日付を求める
    d_today = datetime.date.today()
    # 今日の曜日を求める
    todayweek = datetime.date.today().weekday()
    # print(d_today, todayweek)

    # オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    # 日付は来週の月曜日付となる。
    if (todayweek == 0):  # 今日が月曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today
    elif (todayweek == 1):  # 今日が火曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today - datetime.timedelta(days=1)
    elif (todayweek == 2):  # 今日が水曜日
        Oriconday = d_today - datetime.timedelta(days=2)
    elif (todayweek == 3):  # 今日が木曜日
        Oriconday = d_today - datetime.timedelta(days=3)
    elif (todayweek == 4):  # 今日が金曜日
        Oriconday = d_today - datetime.timedelta(days=4)
    elif (todayweek == 5):  # 今日が土曜日
        Oriconday = d_today - datetime.timedelta(days=5)
    elif (todayweek == 6):  # 今日が日曜日
        Oriconday = d_today - datetime.timedelta(days=6)

    return Oriconday


def OriconSelectWeek():
    # 日付の入力を促す
    date = input("2020年8月3日以降の日付を入力してください (YYYY-MM-DD): ")

    # 入力された日付をdatetimeオブジェクトに変換
    dt = datetime.datetime.strptime(date, "%Y-%m-%d")
    dt = dt.date()
    # 曜日を取得
    weekday = dt.weekday()

    # オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    # 日付は来週の月曜日付となる。
    if (weekday == 0):  # 今日が月曜日
        Oriconday = dt
    elif (weekday == 1):  # 今日が火曜日
        Oriconday = dt - datetime.timedelta(days=1)
    elif (weekday == 2):  # 今日が水曜日
        Oriconday = dt - datetime.timedelta(days=2)
    elif (weekday == 3):  # 今日が木曜日
        Oriconday = dt - datetime.timedelta(days=3)
    elif (weekday == 4):  # 今日が金曜日
        Oriconday = dt - datetime.timedelta(days=4)
    elif (weekday == 5):  # 今日が土曜日
        Oriconday = dt - datetime.timedelta(days=5)
    elif (weekday == 6):  # 今日が日曜日
        Oriconday = dt - datetime.timedelta(days=6)

    return Oriconday


def OriconWeekRank(Oriconday):#オリコン週間ランキング

    #1位から10位
    load_url = "https://www.oricon.co.jp/rank/js/w/" + str(Oriconday) + "/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2',class_='title') #曲名
    artist = soup.find(class_="content-rank-main").find_all('p',class_='name') #アーティスト名
    score = 6.0 #独自スコア
    rank = 1 #ランキング
    print(str(Oriconday) + "付けオリコン週間シングルランキング")

    for link, artist in zip(links, artist):

        if rank != 10:#10位以下（１ケタの場合）なら（点数の位置を揃えるため）
            print("第" + str(rank) + "位 " + " {:.1f}　 ".format(score) + link.text + "/" + artist.text)
        else: #10位（２ケタの場合）なら
            print("第" + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        rank = rank + 1
        score = score - 0.3


    #11位から20位
    load_url = "https://www.oricon.co.jp/rank/js/w/" + str(Oriconday) + "/p/2/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')  # 曲名
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    for link, artist in zip(links, artist):
         print("第" + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
         rank = rank + 1
         score = score - 0.3



def OriconDigitalRank(Oriconday):#オリコンデジタルシングルランキング
    # 1位から10位
    load_url = "https://www.oricon.co.jp/rank/dis/w/" + str(Oriconday) + "/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    rank = 1
    score = float(6.0)
    print(str(Oriconday) + "付けオリコン週間デジタルシングルランキング")
    for link, artist in zip(links, artist):
        if rank != 10:
            print("第" + str(rank) + "位 " + " {:.1f}　 ".format(score) + link.text + "/" + artist.text)
        else:
            print("第" + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        rank = rank + 1
        score = score - 0.3

    # 11位から20位
    load_url = "https://www.oricon.co.jp/rank/dis/w/" + str(Oriconday) + "/p/2/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    for link, artist in zip(links, artist):
        print("第" + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        rank = rank + 1
        score = score - 0.3

sel = int(selecter())

if sel==1: #今週のランキング表示
    OriconTodays()
    OriconWeekRank(OriconTodays())
    print()
    OriconDigitalRank(OriconTodays())

elif sel == 2: #前週のランキング表示
    OriconLastWeek()
    OriconWeekRank(OriconLastWeek())
    print()
    OriconDigitalRank(OriconLastWeek())

else : #別の週のランキング
    OriconWeekRank(OriconSelectWeek())
    print()
    OriconDigitalRank(OriconSelectWeek())

見たい週を選択してください
1:今回 2:前回 3:別の回1
2023-10-30付けオリコン週間シングルランキング
第1位  6.0　 承認欲求/櫻坂46
第2位  5.7　 Carnival/MAZZEL
第3位  5.4　 渚サイコー!/NMB48
第4位  5.1　 アイドルなんかじゃなかったら/AKB48
第5位  4.8　 あのさ、いや別に…/NGT48
第6位  4.5　 想わせぶりっこ/≠ME
第7位  4.2　 TAG ME(HANA_花)/INI
第8位  3.9　 ブルーバード 〜冒険者達へ〜/O2
第9位  3.6　 天体図/サンドリオン
第10位 3.3　 追想のラグナロク/藍月なくる&棗いつき
第11位 3.0　 初心LOVE(うぶらぶ)/なにわ男子
第12位 2.7　 ラブ・ユー・テンダー!/内田真礼
第13位 2.4　 THE IDOLM@STER SHINY COLORS Song for Prism 星の声/シャイニーカラーズ
第14位 2.1　 だってめぐり逢えたんだ/純烈
第15位 1.8　 Dangerholic/Snow Man
第16位 1.5　 好きになっちゃった/SKE48
第17位 1.2　 JUNGLE FIRE feat.MOTSU/芹澤優
第18位 0.9　 オレンジkiss/Snow Man
第19位 0.6　 おひとりさま天国/乃木坂46
第20位 0.3　 ハッピーエンドプリンセス/上坂すみれ

2023-10-30付けオリコン週間デジタルシングルランキング
第1位  6.0　 唱/Ado
第2位  5.7　 SWITCHing/IMP.
第3位  5.4　 勇者/YOASOBI
第4位  5.1　 花/藤井風
第5位  4.8　 Carnival/MAZZEL
第6位  4.5　 JUST DANCE!/Travis Japan
第7位  4.2　 SPECIALZ/King Gnu
第8位  3.9　 アイドル/YOASOBI
第9位  3.6　 クラクラ/Ado
第10位 3.3　 JUNGLE FIRE (feat. MOTSU)/芹澤優
第11位 3.0　 TOO MUCH [Explicit]/The Kid LAROI, Jung Kook, Central Cee
第12位 2.7　 オ